In [1]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window
import talib

In [2]:
global DoEval #是否進行模型評估

#DoEval = False
#next_date_range = ["20180521", "20180522", "20180523", "20180524", "20180525"] #設定預測區間
DoEval = True
#next_date_range = ["20180514", "20180515", "20180516", "20180517", "20180518"] #設定預測區間
ignore_dates = [""]#設定排除日(如端午節)
next_date_range = ["20180507", "20180508", "20180509", "20180510", "20180511"] #設定預測區間
predict_start_date = next_date_range[0]

if sc.master[0:5]=="local":
    Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180518"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180511"
    #Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"
else:
    Path = "hdfs://master:9000/user/hduser"
# Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"

In [3]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [4]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [5]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19305
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [6]:
#取出etf的distinct id
etf_ids = []
etf_id_dic = {}
etf_id_idx = 0
for row in tetfp_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
    etf_id_dic.update({row["etf_id"]: etf_id_idx})
    etf_id_idx += 1
etf_ids

['0050   ',
 '0051   ',
 '0052   ',
 '0053   ',
 '0054   ',
 '0055   ',
 '0056   ',
 '0057   ',
 '0058   ',
 '0059   ',
 '006201 ',
 '006203 ',
 '006204 ',
 '006208 ',
 '00690  ',
 '00692  ',
 '00701  ',
 '00713  ']

In [7]:
etf_dic = {}
for etfid in etf_ids:
    export_dt = tetfp_dt.filter("etf_id='" +etfid+ "' and etf_date < '" + predict_start_date + "'") \
        .orderBy("etf_id", "etf_date", ascending=True)
    export_pd = export_dt.toPandas()
    etf_dic.update({etfid.strip(): export_pd})

In [8]:
#functions of TA lib
print(talib.get_functions())

['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR',

In [9]:
etf_dic["0050"]["etf_close"].count()

1307

In [10]:
import math
def get_feature_pre(curr_list):
    #在feature前加上一個Nan後回傳, 第二個值則回傳原本的最後一個值作為下次的feature
    if type(curr_list) is pd.Series:
        rtn_list = curr_list.tolist()
    else:
        rtn_list = curr_list
    rtn_list = [None] + rtn_list
    return (rtn_list[:-1], [rtn_list[-1]])

def checkNan(num):
    if num == None:
        return True
    elif math.isnan(num):
        return True
    else:
        return False
    
def calculate_n_bias(close_price, ma):
#     (close price - MA)/MA
    if checkNan(close_price) or checkNan(ma):
        return None
    else:
        return (close_price - ma)/ma
    
def calculate_n_bias_list(close_ser, ma_list):
    #calculate n bias (20 days)
    bias_list = []
    for cprice, ma in zip(close_ser.tolist(), ma_list):
        bias_list.append(calculate_n_bias(cprice, ma))
    return bias_list

def calculate_bias_ma_list(ma3_list, ma6_list):
    #calculate bias_ma = (ma3-ma6)/ma6
    bias_list = []
    for ma3, ma6 in zip(ma3_list, ma6_list):
        bias_list.append(calculate_n_bias(ma3, ma6))
    return bias_list

def calculate_bias_3_6_list(bias3_list, bias6_list):
    #bias_3_6 = bias3 - bias6
    bias_list = []
    for bias3, bias6 in zip(bias3_list, bias6_list):
        if checkNan(bias3) or checkNan(bias6):
            bias_list.append(None)
        else:
            bias_list.append(bias3 - bias6)
    return bias_list

def calculate_dif(ema12_list, ema26_list):
    #差離值DIF = 12日EMA - 26日EMA 
    dif_list = []
    for ema12, ema26 in zip(ema12_list, ema26_list):
        if checkNan(ema12) or checkNan(ema26):
            dif_list.append(None)
        else:
            dif_list.append(ema12 - ema26)
    return dif_list

def create_feature(etf_pd, next_date):
    close_ser = etf_pd["etf_close"]
    #EMA
    ema5 = talib.EMA(close_ser,timeperiod=5)
    ema10 = talib.EMA(close_ser,timeperiod=10)
    ema20 = talib.EMA(close_ser,timeperiod=20)
    ema30 = talib.EMA(close_ser,timeperiod=30)
    #BIAS
    #nBIAS -3, 6, 20 => (close price - MA)/MA   ,Paper 建議用20日MA
    ma3 = talib.MA(np.array(close_ser), timeperiod=3)
    ma6 = talib.MA(np.array(close_ser), timeperiod=6)
    ma20 = talib.MA(np.array(close_ser), timeperiod=20)
    nbias3 = calculate_n_bias_list(close_ser, ma3)
    nbias6 = calculate_n_bias_list(close_ser, ma6)
    nbias20 = calculate_n_bias_list(close_ser, ma20)
    
    #BIAS_ma = (ma3-ma6)/ma6
    bias_ma = calculate_bias_ma_list(ma3, ma6)
    
    #bias_3_6 = bias3 - bias6
    bias_3_6 = calculate_bias_3_6_list(nbias3, nbias6)
    
    #KD --> only use STOCHRSI
    k, d = talib.STOCHRSI(close_ser, timeperiod=9, fastk_period=3, fastd_period=3, fastd_matype=1)
    
    #差離值DIF = 12日EMA - 26日EMA 
    ema12 = talib.EMA(close_ser,timeperiod=12)
    ema26 = talib.EMA(close_ser,timeperiod=26)
    dif = calculate_dif(ema12, ema26)
    
    #MACD
    macd, macdsignal, macdhist = talib.MACD(close_ser, fastperiod=12, slowperiod=26, signalperiod=9)
    
    #RSI
    rsi = talib.RSI(close_ser, timeperiod=10)
    
    #CMO
    cmo = talib.CMO(close_ser, timeperiod=14)
    
    #ROCR - Rate of change ratio: (price/prevPrice)
    rocr = talib.ROCR(close_ser, timeperiod=10)
    
    #PPO - Percentage Price Oscillator
    ppo = talib.PPO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    
    #作出目前的feature table及預測用的feature table
    ema5, next_ema5 = get_feature_pre(ema5)
    ema10, next_ema10 = get_feature_pre(ema10)
    ema20, next_ema20 = get_feature_pre(ema20)
    ema30, next_ema30 = get_feature_pre(ema30)
    
    nbias3, next_nbias3 = get_feature_pre(nbias3)
    nbias6, next_nbias6 = get_feature_pre(nbias6)
    nbias20, next_nbias20 = get_feature_pre(nbias20)
    
    bias_ma, next_bias_ma = get_feature_pre(bias_ma)
    bias_3_6, next_bias_3_6 = get_feature_pre(bias_3_6)
    k, next_k = get_feature_pre(k)
    d, next_d = get_feature_pre(d)
    dif, next_dif = get_feature_pre(dif)
    
    macd, next_macd = get_feature_pre(macd)
    macdsignal, next_macdsignal = get_feature_pre(macdsignal)
    macdhist, next_macdhist = get_feature_pre(macdhist)
    
    rsi, next_rsi = get_feature_pre(rsi)
    cmo, next_cmo = get_feature_pre(cmo)
    rocr, next_rocr = get_feature_pre(rocr)
    ppo, next_ppo = get_feature_pre(ppo)
    
    etfid = etf_pd["etf_id"].tolist()[0]
    rtn_pd = pd.DataFrame({"etf_id": etfid, "etf_date": etf_pd["etf_date"], "etf_close": etf_pd["etf_close"], 
                           "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                          "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          "macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo})
    next_pd = pd.DataFrame({"etf_id": etfid, "etf_date": next_date, "etf_close": -1, 
                           "ema5": next_ema5, "ema10": next_ema10, "ema20": next_ema20, "ema30" : next_ema30,
                          "nbias3": next_nbias3, "nbias6": next_nbias6, "nbias20": next_nbias20, "bias_ma": next_bias_ma,
                          "bias_3_6": next_bias_3_6, "k": next_k, "d": next_d, "dif": next_dif,
                          "macd": next_macd, "macdsignal": next_macdsignal, "macdhist": next_macdhist,
                          "rsi": next_rsi, "cmo": next_cmo, "rocr": next_rocr, "ppo": next_ppo})
 
    return (rtn_pd, next_pd)

In [11]:
val = get_feature_pre(etf_dic["0050"]["etf_close"])
print(val[1])


[78.95]


In [12]:
#EMA
etf_pd_dic = {}
next_pd_dic = {}
for etfid in etf_ids:
    etf_pd, next_pd = create_feature(etf_dic[etfid.strip()], predict_start_date)
    etf_pd_dic.update({etfid.strip() : etf_pd.dropna(how='any')})
    next_pd_dic.update({etfid.strip() : next_pd})

In [42]:
#作出訓練集
etfids_1 = ["0050", "0051", "0052", "0053", "0057", "0058", "006201", "006203", "006204", "006208"]
etfids_2 = ["00690","00692", "00701", "00713"]
etfids_3 = [etfid.strip() for etfid in etf_ids if etfid.strip() not in etfids_1 and etfid.strip() not in etfids_2 ]

train_dt1 = pd.concat([etf_pd_dic[etfid] for etfid in etfids_1 + etfids_3 + etfids_2])
print('train_dt1 count:', train_dt1.shape)
train_dt2 = pd.concat([etf_pd_dic[etfid] for etfid in etfids_2])
print('train_dt2 count:', train_dt2.shape)


train_dt1 count: (18513, 22)
train_dt2 count: (691, 22)


In [15]:
non_feature_list = ['etf_id', 'etf_date', 'etf_close']
list(train_dt1.columns.values)

['bias_3_6',
 'bias_ma',
 'cmo',
 'd',
 'dif',
 'ema10',
 'ema20',
 'ema30',
 'ema5',
 'etf_close',
 'etf_date',
 'etf_id',
 'k',
 'macd',
 'macdhist',
 'macdsignal',
 'nbias20',
 'nbias3',
 'nbias6',
 'ppo',
 'rocr',
 'rsi']

In [43]:
import xgboost as xgb
feature_cols = [col for col in list(train_dt1.columns.values) if col not in non_feature_list]
#$$$ parameter
rf_model1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
        colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0)
train_x1 = train_dt1[feature_cols].values
train_y1 = train_dt1["etf_close"].values
rf_model1.fit(train_x1, train_y1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.08, max_delta_step=0,
       max_depth=68, min_child_weight=0, missing=None, n_estimators=100,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [65]:
train_dt1.loc[(train_dt1["etf_id"] == "0050   ") & (train_dt1["etf_date"] < predict_start_date) ].iloc[-1]["etf_close"]
next_pd_dic["0050"]["etf_close"] = -2
etf_dic["0050"]

,etf_id,etf_date,etf_name,etf_open,etf_high,etf_low,etf_close,etf_count
0,00713,20170927,元大台灣高息低波,29.70,29.93,29.70,29.880000,682.0
1,00713,20170928,元大台灣高息低波,29.90,29.95,29.82,29.840000,716.0
2,00713,20170929,元大台灣高息低波,29.84,29.90,29.60,29.800000,541.0
3,00713,20170930,元大台灣高息低波,29.84,29.95,29.84,29.920000,115.0
4,00713,20171002,元大台灣高息低波,29.94,30.11,29.93,30.100000,288.0
5,00713,20171003,元大台灣高息低波,30.10,30.42,30.10,30.400000,413.0
6,00713,20171005,元大台灣高息低波,30.40,30.80,30.40,30.700000,3507.0
7,00713,20171006,元大台灣高息低波,30.71,30.71,30.50,30.600000,110.0
8,00713,20171011,元大台灣高息低波,30.70,30.81,30.63,30.750000,3388.0
9,00713,20171012,元大台灣高息低波,30.74,30.75,30.66,30.720000,181.0


In [45]:
#計算上或下的值(udf)
def judge_up_down_pred_native(curr_price, prev_close_price):
    if checkNan(prev_close_price):
        return 0.0
    else:
        prev_price = prev_close_price
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0

In [46]:
next_date_range[-1]

'20180511'

In [64]:
predict_res_final = []
for date in next_date_range:
    for eid in etf_ids:
        #do predict
        pred_data = next_pd_dic[eid.strip()][feature_cols].values
        pred_res = rf_model1.predict(data=pred_data)
        print(pred_res)
        prediction = pred_res[0]
        
        etf_dt = etf_dic[etfid.strip()]
        prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
#         print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
        prev_close = prev_dt["etf_close"]
        #取出所有預測結果作合併，以進行後續成績計算
        predict_res_final.append({
            'etf_id': eid, 'etf_date': date,
            'prediction': prediction, 'pred_ud': judge_up_down_pred_native(pred_res, prev_close)
        })
        
        if date != next_date_range[-1]:
            #作出新的next_pd
            new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                       'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
            new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                      'etf_low', 'etf_close', 'etf_count']]
            #print(new_pd)
            etf_dt=etf_dt.append(new_pd, ignore_index=True)
            next_date = next_date_range[next_date_range.index(date)+1]
            etf_pd2, next_pd2 = create_feature(etf_dt, next_date)
            etf_dic.update({eid.strip(): etf_dt})
            next_pd_dic.update({eid.strip(): next_pd2})

pd.DataFrame(predict_res_final)

[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.45960999]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.32586098]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.28718185]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327522]
[ 30.18327

,etf_date,etf_id,pred_ud,prediction
0,20180507,0050,1.0,30.459610
1,20180507,0051,1.0,30.459610
2,20180507,0052,1.0,30.459610
3,20180507,0053,1.0,30.459610
4,20180507,0054,1.0,30.459610
5,20180507,0055,1.0,30.459610
6,20180507,0056,1.0,30.459610
7,20180507,0057,1.0,30.459610
8,20180507,0058,1.0,30.459610
9,20180507,0059,1.0,30.459610
